### Importing packages

In [1]:
# ! pip install -U pandas lxml networkx

In [3]:
import os
import sys
import re
import pickle
import json

import pandas as pd
import matplotlib.pyplot as plt

import xml.etree.ElementTree as ET
from lxml import etree
from bs4 import BeautifulSoup as bs
from bs4 import NavigableString

sys.path.insert(1, '../scripts/utils/')
from text_filter import spacify_soup

In [4]:
path = '/home/jovyan/shared/C_amc_141/R_amc_3.1_12921/203_vert_spacy_rftt/'

### Retrieving pickle

In [5]:
df_files = pd.read_pickle('../outputs/df_files.pkl')

In [6]:
df_files.head()

,path,name,extension,size,atime,mtime,ctime,folder,depth,parent,uid,main,source,year,date
0,artikel_/APA/1986/APA_19860220.xml,APA_19860220,xml,6142251,2020-04-05 18:21:09,2019-09-20 08:04:05,2019-09-23 11:43:22,False,4,5,1005,artikel_,APA,1986,1986-02-20
1,artikel_/APA/1986/APA_19860102.xml,APA_19860102,xml,3397140,2020-02-13 14:35:33,2019-09-20 08:04:05,2019-09-23 11:43:22,False,4,5,1005,artikel_,APA,1986,1986-01-02
2,artikel_/APA/1986/APA_19860222.xml,APA_19860222,xml,3352934,2020-02-13 14:35:33,2019-09-20 08:04:05,2019-09-23 11:43:22,False,4,5,1005,artikel_,APA,1986,1986-02-22
3,artikel_/APA/1986/APA_19860103.xml,APA_19860103,xml,3794819,2020-02-13 14:35:33,2019-09-20 08:04:06,2019-09-23 11:43:22,False,4,5,1005,artikel_,APA,1986,1986-01-03
4,artikel_/APA/1986/APA_19860221.xml,APA_19860221,xml,5763730,2020-02-13 14:35:33,2019-09-20 08:04:06,2019-09-23 11:43:22,False,4,5,1005,artikel_,APA,1986,1986-02-21


### Examining a single file:

In [121]:
filepath = os.path.join(path, df_files.loc[1, 'path'])
print(filepath)

/home/jovyan/shared/C_amc_141/R_amc_3.1_12921/203_vert_spacy_rftt/artikel_/APA/1986/APA_19860102.xml


In [122]:
with open(filepath, "r") as f:
    soup = bs(f, "lxml-xml")

In [124]:
soup

In [148]:
headers = soup.find_all('doc')
print("number of headers: ", len(headers))

print('-----')
for headers in headers[0:3]:
    for key in headers.attrs.keys():
        print(f"{key}: {headers[key]}")
    print('-----')

number of headers:  196
-----
autor: je
region: agesamt
datum: 1986-01-02
ressort2: ausland inland
tokens: 176
keys: JA_1986 AG_APA RS_AI RS_A DA_19860102 MO_198601 DB_APA
id: APA_19860102_APA0001
mediatype: print
datum_full: 1986-01-02T00:00:00Z
docsrc: APA
docsrc_name: APA-Meldungen digital
bibl: APA-Meldungen digital  vom 1986-01-02
-----
autor: je
region: agesamt
datum: 1986-01-02
ressort2: ausland
tokens: 147
keys: JA_1986 AG_APA RS_AA RS_A DA_19860102 MO_198601 DB_APA
id: APA_19860102_APA0002
mediatype: print
datum_full: 1986-01-02T00:00:00Z
docsrc: APA
docsrc_name: APA-Meldungen digital
bibl: APA-Meldungen digital  vom 1986-01-02
-----
autor: je
region: agesamt
datum: 1986-01-02
ressort2: ausland
tokens: 89
keys: JA_1986 AG_APA RS_AA RS_A DA_19860102 MO_198601 DB_APA
id: APA_19860102_APA0003
mediatype: print
datum_full: 1986-01-02T00:00:00Z
docsrc: APA
docsrc_name: APA-Meldungen digital
bibl: APA-Meldungen digital  vom 1986-01-02
-----


In [152]:
docs = soup.find_all('field')
print("number of docs: ", len(docs))

#print('-----')
for doc in docs[0:10]:
    for key in doc.attrs.keys():
        print(f"{key}: {doc[key]}")
print('-----')

stichwort = soup.find_all(attrs={"name" : "stichwort"})
print(f'{len(stichwort)} stichwort elements')
titel = soup.find_all(attrs={"name" : "titel"})
print(f'{len(titel)} titel elements')
inhalt = soup.find_all(attrs={"name" : "inhalt"})
print(f'{len(inhalt)} inhalt elements')

number of docs:  832
name: stichwort
name: stichwort
name: stichwort
name: titel
name: inhalt
name: stichwort
name: stichwort
name: stichwort
name: titel
name: inhalt
-----
440 stichwort elements
196 titel elements
196 inhalt elements


In [184]:
output = stichwort[0].find_all('s')
for idn, out in enumerate(output):
    print(f'\n--------- text {idn} ------------')
    text = re.sub('<g/>','',out.text)
    text = re.sub(r'[\n]+','\n',text)
    print(text)
    print('\n+++ rows +++\n')
    rows = re.split('\n',text)[1:-1]
    print(rows)
    print('\n+++ fields +++\n')
    for row in rows:
        fields = re.split('\t',row)
        print(len(fields[:-1]), fields[:-1])
print('------------')


--------- text 0 ------------

Österreich	österreich	0	PROPN	NE	LOC	B	0	ROOT	0	-	-	N.Name.Gen.Sg.Neut	NE	Österreich-n	tt


+++ rows +++

['Österreich\tösterreich\t0\tPROPN\tNE\tLOC\tB\t0\tROOT\t0\t-\t-\tN.Name.Gen.Sg.Neut\tNE\tÖsterreich-n\ttt']

+++ fields +++

15 ['Österreich', 'österreich', '0', 'PROPN', 'NE', 'LOC', 'B', '0', 'ROOT', '0', '-', '-', 'N.Name.Gen.Sg.Neut', 'NE', 'Österreich-n']
------------


In [181]:
output = titel[1].find_all('s')
for idn, out in enumerate(output):
    print(f'\n--------- text {idn} ------------')
    text = re.sub('<g/>','',out.text)
    text = re.sub(r'[\n]+','\n',text)
    print(text)
    print('\n+++ rows +++\n')
    rows = re.split('\n',text)[1:-1]
    print(rows)
    print('\n+++ fields +++\n')
    for row in rows:
        fields = re.split('\t',row)
        print(len(fields[:-1]), fields[:-1])
print('------------')


--------- text 0 ------------

Reagan	reagan	0	PROPN	NE	PERSON	B	0	sb	1	-	-	N.Name.Nom.Sg.Masc	NE	Reagan-n	tt
begrüßt	begrüßt	1	VERB	VVFIN		O	-	ROOT	1	-	begrüßen	VFIN.Full.3.Sg.Pres.Ind	VVFIN	begrüßen-v	tt
Gorbatschows	gorbatschows	2	PROPN	NE	PERSON	B	2	ag	3	-	-	N.Name.Gen.Sg.Masc	NE	Gorbatschow-n	tt
Neujahrsbotschaft	neujahrsbotschaft	3	NOUN	NN		O	2	oa	1	-	-	N.Reg.Acc.Sg.Fem	NN	Neujahrsbotschaft-n	tt
2	2	4	NUM	CARD		O	-	nk	3	-	-	CARD	CARD	2-m	tt


+++ rows +++

['Reagan\treagan\t0\tPROPN\tNE\tPERSON\tB\t0\tsb\t1\t-\t-\tN.Name.Nom.Sg.Masc\tNE\tReagan-n\ttt', 'begrüßt\tbegrüßt\t1\tVERB\tVVFIN\t\tO\t-\tROOT\t1\t-\tbegrüßen\tVFIN.Full.3.Sg.Pres.Ind\tVVFIN\tbegrüßen-v\ttt', 'Gorbatschows\tgorbatschows\t2\tPROPN\tNE\tPERSON\tB\t2\tag\t3\t-\t-\tN.Name.Gen.Sg.Masc\tNE\tGorbatschow-n\ttt', 'Neujahrsbotschaft\tneujahrsbotschaft\t3\tNOUN\tNN\t\tO\t2\toa\t1\t-\t-\tN.Reg.Acc.Sg.Fem\tNN\tNeujahrsbotschaft-n\ttt', '2\t2\t4\tNUM\tCARD\t\tO\t-\tnk\t3\t-\t-\tCARD\tCARD\t2-m\ttt']

+++ fields +++

15 [

In [182]:
inhalt = soup.find_all(attrs={"name" : "inhalt"})
print(f'{len(inhalt)} inhalt elements')
output = inhalt[0].find_all('s')
for idn, out in enumerate(output[0:3]):
    print(f'\n--------- text {idn} ------------')
    text = re.sub('<g/>','',out.text)
    text = re.sub(r'[\n]+','\n',text)
    print(text)
    print('\n+++ rows +++\n')
    rows = re.split('\n',text)[1:-1]
    print(rows)
    print('\n+++ fields +++\n')
    for row in rows:
        fields = re.split('\t',row)
        print(len(fields[:-1]), fields[:-1])
print('------------')

196 inhalt elements

--------- text 0 ------------

Utl	utl	0	NOUN	NN		O	0	ROOT	0	-	-	FM	NE	Utl-n	u
.	.	1	PUNCT	$.		O	-	punct	0	-	-	SYM.Pun.Sent	$.	.-x	tt


+++ rows +++

['Utl\tutl\t0\tNOUN\tNN\t\tO\t0\tROOT\t0\t-\t-\tFM\tNE\tUtl-n\tu', '.\t.\t1\tPUNCT\t$.\t\tO\t-\tpunct\t0\t-\t-\tSYM.Pun.Sent\t$.\t.-x\ttt']

+++ fields +++

15 ['Utl', 'utl', '0', 'NOUN', 'NN', '', 'O', '0', 'ROOT', '0', '-', '-', 'FM', 'NE', 'Utl-n']
15 ['.', '.', '1', 'PUNCT', '$.', '', 'O', '-', 'punct', '0', '-', '-', 'SYM.Pun.Sent', '$.', '.-x']

--------- text 1 ------------

:	:	2	PUNCT	$.		O	-	ROOT	2	-	-	SYM.Pun.Colon	$.	:-x	tt


+++ rows +++

[':\t:\t2\tPUNCT\t$.\t\tO\t-\tROOT\t2\t-\t-\tSYM.Pun.Colon\t$.\t:-x\ttt']

+++ fields +++

15 [':', ':', '2', 'PUNCT', '$.', '', 'O', '-', 'ROOT', '2', '-', '-', 'SYM.Pun.Colon', '$.', ':-x']

--------- text 2 ------------

Altkanzler	altkanzler	3	NOUN	NN		O	0	ROOT	3	-	Altkanzler	N.Reg.Nom.Sg.Masc	NN	Altkanzler-n	tt
im	im	4	ADP	APPRART		O	-	mnr	3	-	-	APPRART.Dat.Sg.Masc	

In [196]:
for doc in docs[0:5]:
    print(f"name: {doc['name']}")
    for d in doc.find_all('s'):
        text = re.sub('<g/>','',d.text)
        text = re.sub(r'[\n]+','\n',text)
        rows = re.split('\n',text)[1:-1]
        for row in rows:
            fields = re.split('\t',row)
            print(len(fields[:-1]), fields[:-1])
    print('------------')

name: stichwort
15 ['Österreich', 'österreich', '0', 'PROPN', 'NE', 'LOC', 'B', '0', 'ROOT', '0', '-', '-', 'N.Name.Gen.Sg.Neut', 'NE', 'Österreich-n']
------------
name: stichwort
15 ['Terror', 'terror', '0', 'NOUN', 'NN', '', 'O', '0', 'ROOT', '0', '-', 'Terror', 'N.Reg.Nom.Sg.Masc', 'NN', 'Terror-n']
------------
name: stichwort
15 ['Kreisky', 'kreisky', '0', 'PROPN', 'NE', 'PERSON', 'B', '0', 'ROOT', '0', '-', '-', 'N.Name.Nom.Sg.Masc', 'NE', 'Kreisky-n']
------------
name: titel
15 ['Kreisky', 'kreisky', '0', 'PROPN', 'NE', 'PERSON', 'B', '0', 'ROOT', '0', '-', '-', 'N.Name.Nom.Sg.Masc', 'NE', 'Kreisky-n']
15 [':', ':', '1', 'PUNCT', '$.', '', 'O', '-', 'punct', '0', '-', '-', 'SYM.Pun.Colon', '$.', ':-x']
15 ['Ursachen', 'ursachen', '2', 'NOUN', 'NN', '', 'O', '-', 'rs', '0', '-', 'Ursache', 'N.Reg.Nom.Pl.Fem', 'NN', 'Ursache-n']
15 ['des', 'des', '3', 'DET', 'ART', '', 'O', '1', 'nk', '4', '-', '-', 'ART.Def.Gen.Sg.Masc', 'ART', 'die-x']
15 ['Terrors', 'terrors', '4', 'NOUN', 'N